In [1]:
from LorisBallsBasedModel import *
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']
cars = pd.read_csv(r'data\car_evaluation_dataset\car_evaluation.csv', names=cols, header=None)
cars.head(3)

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc


In [3]:
labels = pd.get_dummies(cars.output, prefix='condition')

X = cars[['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']]

y = labels.values

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# SingleLayerPerceptron base model

In [5]:
X_train_embedding_layers = {}
for input_name in X_train.columns:
    X_train_embedding_layers[input_name] = StringEmbedding(X_train[input_name].unique(), len(X_train[input_name].unique()))

processing_layer = InputsProcessing(X_train_embedding_layers)
    
X_train_dict = X_train.to_dict(orient='list')
X_test_dict = X_test.to_dict(orient='list')

train_tensor = tf.data.Dataset.from_tensor_slices((X_train_dict,
                                                   y_train))
train_tensor = train_tensor.cache().shuffle(300).batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [6]:
SLP = SingleLayerPerceptron(tf.keras.layers.Dense(4, activation='softmax'),
                            processing_layer=processing_layer)
SLP.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [7]:
history = SLP.fit(train_tensor, epochs=50, verbose=1)

Epoch 1/50
173/173 [==============================] - 2s 2ms/step - loss: 1.1130 - acc: 0.6816
Epoch 2/50
173/173 [==============================] - 0s 2ms/step - loss: 0.7720 - acc: 0.7055
Epoch 3/50
173/173 [==============================] - 0s 2ms/step - loss: 0.6205 - acc: 0.7106
Epoch 4/50
173/173 [==============================] - 0s 2ms/step - loss: 0.5097 - acc: 0.7988
Epoch 5/50
173/173 [==============================] - 0s 2ms/step - loss: 0.4375 - acc: 0.8466
Epoch 6/50
173/173 [==============================] - 0s 2ms/step - loss: 0.3889 - acc: 0.8618
Epoch 7/50
173/173 [==============================] - 0s 2ms/step - loss: 0.3516 - acc: 0.8654
Epoch 8/50
173/173 [==============================] - 0s 2ms/step - loss: 0.3232 - acc: 0.8705
Epoch 9/50
173/173 [==============================] - 0s 1ms/step - loss: 0.3009 - acc: 0.8813
Epoch 10/50
173/173 [==============================] - 0s 2ms/step - loss: 0.2818 - acc: 0.8907
Epoch 11/50
173/173 [============================

In [8]:
test_tensor = tf.data.Dataset.from_tensor_slices((X_test_dict,
                                                  y_test))
test_tensor = test_tensor.cache().shuffle(300).batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

SLP.evaluate(test_tensor)

44/44 [==============================] - 0s 1ms/step - loss: 0.1636 - acc: 0.9191


[0.16359318792819977, 0.9190751314163208]

# Transfert learning

In [9]:
X_train_embedding_layers_1 = {}
for input_name in X_train.columns:
    X_train_embedding_layers_1[input_name] = StringEmbedding(X_train[input_name].unique(), len(X_train[input_name].unique()))

processing_layer_1 = InputsProcessing(X_train_embedding_layers_1)

LBBM_1 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              processing_layer=processing_layer_1,
                              base_model=SLP)
LBBM_1.freeze_base_model()
LBBM_1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [10]:
history_LBBM_1 = LBBM_1.fit(train_tensor, epochs=50, verbose=1)

Epoch 1/50
173/173 [==============================] - 2s 3ms/step - loss: 1.0206 - acc: 0.6903
Epoch 2/50
173/173 [==============================] - 0s 3ms/step - loss: 0.8219 - acc: 0.7055
Epoch 3/50
173/173 [==============================] - 0s 3ms/step - loss: 0.7282 - acc: 0.7294
Epoch 4/50
173/173 [==============================] - 0s 3ms/step - loss: 0.6306 - acc: 0.8111
Epoch 5/50
173/173 [==============================] - 0s 3ms/step - loss: 0.5392 - acc: 0.8640
Epoch 6/50
173/173 [==============================] - 0s 3ms/step - loss: 0.4670 - acc: 0.8734
Epoch 7/50
173/173 [==============================] - 0s 3ms/step - loss: 0.4204 - acc: 0.8777
Epoch 8/50
173/173 [==============================] - 0s 3ms/step - loss: 0.3897 - acc: 0.8763
Epoch 9/50
173/173 [==============================] - 0s 3ms/step - loss: 0.3608 - acc: 0.8770
Epoch 10/50
173/173 [==============================] - 0s 3ms/step - loss: 0.3368 - acc: 0.8748
Epoch 11/50
173/173 [============================

In [11]:
LBBM_1.evaluate(test_tensor)

44/44 [==============================] - 7s 7ms/step - loss: 0.1073 - acc: 0.9566


[0.10727071762084961, 0.9566473960876465]

In [12]:
LBBM_1.base_model.evaluate(test_tensor)

44/44 [==============================] - 0s 2ms/step - loss: 0.1573 - acc: 0.9364


[0.15729738771915436, 0.9364162087440491]

In [13]:
LBBM_1.unfreeze_model()
LBBM_1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [14]:
history_LBBM_1 = LBBM_1.fit(train_tensor, epochs=50, verbose=1)

Epoch 1/50
173/173 [==============================] - 42s 18ms/step - loss: 0.0648 - acc: 0.9870
Epoch 2/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0629 - acc: 0.9891
Epoch 3/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0616 - acc: 0.9891
Epoch 4/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0601 - acc: 0.9877
Epoch 5/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0590 - acc: 0.9899
Epoch 6/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0571 - acc: 0.9906
Epoch 7/50
173/173 [==============================] - 4s 20ms/step - loss: 0.0557 - acc: 0.9913
Epoch 8/50
173/173 [==============================] - 4s 20ms/step - loss: 0.0552 - acc: 0.9891
Epoch 9/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0539 - acc: 0.9920:
Epoch 10/50
173/173 [==============================] - 3s 20ms/step - loss: 0.0529 - acc: 0.9906
Epoch 11/50
173/173 [================

In [15]:
LBBM_1.evaluate(test_tensor)

44/44 [==============================] - 6s 6ms/step - loss: 0.0542 - acc: 0.9798


[0.054222166538238525, 0.9797688126564026]

In [16]:
LBBM_1.base_model.evaluate(test_tensor)

44/44 [==============================] - 0s 866us/step - loss: 0.2373 - acc: 0.9162


[0.23733124136924744, 0.9161849617958069]

## Adding a second BoundedParaboloids layer

In [17]:
X_train_embedding_layers_2 = {}
for input_name in X_train.columns:
    X_train_embedding_layers_2[input_name] = StringEmbedding(X_train[input_name].unique(), len(X_train[input_name].unique()))

processing_layer_2 = InputsProcessing(X_train_embedding_layers_2)

LBBM_2 = LorisBallsBasedModel(BoundedParaboloids(128, semi_axis_regularizer=SemiAxisRegularizer(0.00001)),
                              tf.keras.layers.Dense(4, activation='softmax'),
                              processing_layer=processing_layer_2,
                              base_model=LBBM_1)
LBBM_2.freeze_base_model()
LBBM_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [18]:
history_LBBM_2 = LBBM_2.fit(train_tensor, epochs=50, verbose=1)

Epoch 1/50
173/173 [==============================] - 75s 23ms/step - loss: 1.1206 - acc: 0.6671
Epoch 2/50
173/173 [==============================] - 5s 27ms/step - loss: 0.8722 - acc: 0.7055: 1s - los
Epoch 3/50
173/173 [==============================] - 5s 27ms/step - loss: 0.7474 - acc: 0.7055
Epoch 4/50
173/173 [==============================] - 5s 26ms/step - loss: 0.6464 - acc: 0.7605
Epoch 5/50
173/173 [==============================] - 5s 28ms/step - loss: 0.5296 - acc: 0.8589
Epoch 6/50
173/173 [==============================] - 5s 29ms/step - loss: 0.4140 - acc: 0.9219
Epoch 7/50
173/173 [==============================] - 5s 28ms/step - loss: 0.3377 - acc: 0.9370: 1s - loss:
Epoch 8/50
173/173 [==============================] - 5s 28ms/step - loss: 0.2886 - acc: 0.9703
Epoch 9/50
173/173 [==============================] - 5s 28ms/step - loss: 0.2491 - acc: 0.9776
Epoch 10/50
173/173 [==============================] - 5s 28ms/step - loss: 0.2162 - acc: 0.9913
Epoch 11/50
173/

In [19]:
LBBM_2.evaluate(test_tensor)

44/44 [==============================] - 13s 13ms/step - loss: 0.0710 - acc: 0.9827


[0.07099396735429764, 0.9826589822769165]

In [20]:
LBBM_2.base_model.evaluate(test_tensor)

44/44 [==============================] - 0s 7ms/step - loss: 0.0542 - acc: 0.9798


[0.05422215536236763, 0.9797688126564026]

In [21]:
LBBM_2.unfreeze_model()
LBBM_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['acc'])

In [22]:
history_LBBM_2 = LBBM_2.fit(train_tensor, epochs=50, verbose=1)

Epoch 1/50
173/173 [==============================] - 85s 35ms/step - loss: 0.0361 - acc: 0.9986
Epoch 2/50
173/173 [==============================] - 6s 37ms/step - loss: 0.0359 - acc: 0.9978
Epoch 3/50
173/173 [==============================] - 6s 37ms/step - loss: 0.0339 - acc: 0.9986
Epoch 4/50
173/173 [==============================] - 7s 42ms/step - loss: 0.0325 - acc: 0.9993
Epoch 5/50
173/173 [==============================] - 7s 39ms/step - loss: 0.0324 - acc: 0.9986
Epoch 6/50
173/173 [==============================] - 6s 38ms/step - loss: 0.0324 - acc: 0.9986
Epoch 7/50
173/173 [==============================] - 6s 37ms/step - loss: 0.0307 - acc: 1.0000
Epoch 8/50
173/173 [==============================] - 7s 38ms/step - loss: 0.0303 - acc: 1.0000
Epoch 9/50
173/173 [==============================] - 6s 37ms/step - loss: 0.0300 - acc: 1.0000
Epoch 10/50
173/173 [==============================] - 7s 38ms/step - loss: 0.0300 - acc: 1.0000
Epoch 11/50
173/173 [=================

In [23]:
LBBM_2.evaluate(test_tensor)

44/44 [==============================] - 13s 11ms/step - loss: 0.0291 - acc: 0.9942


[0.029054654762148857, 0.9942196607589722]

In [24]:
LBBM_2.base_model.evaluate(test_tensor)

44/44 [==============================] - 0s 7ms/step - loss: 0.0314 - acc: 0.9942


[0.03144938498735428, 0.9942196607589722]